In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing
import csv
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

pd.options.mode.chained_assignment = None
data = pd.read_csv('StatsAfterAllNBAInsertion.csv')

In [2]:
data.describe()

,Year,Age,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,MPpG,PER,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
count,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15095.000000,...,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000
mean,2004.197550,26.984834,0.027483,0.276358,0.430993,48.339603,23.073907,1130.345695,19.823603,12.399821,...,36.770662,0.642278,23.103245,0.398854,67.158808,1.190523,101.926821,1.864801,461.931854,7.824821
std,7.998757,4.149146,0.163493,1.305299,1.765006,26.478508,28.236547,915.995235,10.293082,6.282646,...,36.014443,0.466536,34.770722,0.497782,62.887391,0.794410,76.944305,0.851711,461.987449,5.834517
min,1990.000000,18.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-90.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1997.000000,24.000000,0.000000,0.000000,0.000000,25.000000,0.000000,292.000000,11.200000,9.600000,...,8.000000,0.300000,3.000000,0.100000,16.000000,0.600000,32.000000,1.200000,86.000000,3.300000
50%,2005.000000,26.000000,0.000000,0.000000,0.000000,52.000000,8.000000,940.000000,18.900000,12.600000,...,27.000000,0.600000,11.000000,0.200000,50.000000,1.000000,92.000000,1.900000,317.000000,6.300000
75%,2011.000000,30.000000,0.000000,0.000000,0.000000,74.000000,43.000000,1835.250000,28.300000,15.600000,...,55.000000,0.900000,28.000000,0.500000,102.000000,1.600000,159.000000,2.500000,713.250000,11.200000
max,2017.000000,44.000000,1.000000,15.000000,15.000000,85.000000,83.000000,3533.000000,44.500000,129.100000,...,246.000000,3.500000,376.000000,6.000000,464.000000,5.700000,371.000000,6.000000,2832.000000,35.400000


In [3]:
# Fill in blanks/NAs

data['PER'] = data['PER'].fillna(data['PER'].median())
data['TS%'] = data['TS%'].fillna(data['TS%'].median())
data['3PAr'] = data['3PAr'].fillna(data['3PAr'].median())
data['FTr'] = data['FTr'].fillna(data['FTr'].median())
data['ORB%'] = data['ORB%'].fillna(data['ORB%'].median())
data['DRB%'] = data['DRB%'].fillna(data['DRB%'].median())
data['TRB%'] = data['TRB%'].fillna(data['TRB%'].median())
data['AST%'] = data['AST%'].fillna(data['AST%'].median())
data['STL%'] = data['STL%'].fillna(data['STL%'].median())
data['BLK%'] = data['BLK%'].fillna(data['BLK%'].median())
data['TOV%'] = data['TOV%'].fillna(data['TOV%'].median())
data['USG%'] = data['USG%'].fillna(data['USG%'].median())
data['WS/48'] = data['WS/48'].fillna(data['WS/48'].median())
data['FG%'] = data['FG%'].fillna(data['FG%'].median())
data['3P%'] = data['3P%'].fillna(data['3P%'].median())
data['2P%'] = data['2P%'].fillna(data['2P%'].median())
data['eFG%'] = data['eFG%'].fillna(data['eFG%'].median())
data['FT%'] = data['FT%'].fillna(data['FT%'].median())


In [4]:
data.Pos.unique()

array(['PF', 'SF', 'C', 'SG', 'PG', 'SG-SF', 'PF-C', 'SF-PF', 'PF-SF',
       'SG-PG', 'PG-SG', 'C-PF', 'C-SF', 'SF-SG', 'PG-SF', 'SG-PF'],
      dtype=object)

In [5]:
#Give Position numerical labels

le = preprocessing.LabelEncoder()
data['Pos'] = le.fit_transform(data['Pos'].astype(str))
le_Pos_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_Pos_mapping)

{'C': 0, 'C-PF': 1, 'C-SF': 2, 'PF': 3, 'PF-C': 4, 'PF-SF': 5, 'PG': 6, 'PG-SF': 7, 'PG-SG': 8, 'SF': 9, 'SF-PF': 10, 'SF-SG': 11, 'SG': 12, 'SG-PF': 13, 'SG-PG': 14, 'SG-SF': 15}


In [6]:
# Delete duplicate Data that occured when Player transferred mid season

check = 0
index = 0
for year in data['Year']:
    if (check != year):
        check = year
        index = index + 1
        continue
    #remove row using index
    data = data.drop(index)
    index = index + 1

In [7]:
# Check if duplicate deletion was successful

def checkForDuplicates(dataframe):
    check = 0
    count = 0
    for year in dataframe['Year']:
        if (count == 2):
            return True
        if (check != year):
            check = year
            count = 0
            continue
        count = count + 1
        continue
    return False

In [8]:
print(checkForDuplicates(data))

False


In [9]:
data.describe()

,Year,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,MPpG,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
count,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,...,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.00000,12219.000000,12219.000000
mean,2003.997217,26.949505,5.904166,0.033718,0.312955,0.496358,54.075211,26.483182,1280.981750,20.566208,...,41.714379,0.666454,26.458384,0.422179,76.029790,1.229888,115.083313,1.91911,526.192978,8.213725
std,7.995671,4.213603,4.284325,0.180510,1.395725,1.904681,24.976332,29.590902,925.777987,10.262107,...,37.332904,0.469495,36.990469,0.508466,64.856059,0.799250,76.999472,0.83143,478.698506,5.949535
min,1990.000000,18.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1997.000000,24.000000,3.000000,0.000000,0.000000,0.000000,35.000000,1.000000,425.500000,12.000000,...,11.000000,0.300000,4.000000,0.100000,22.000000,0.600000,47.000000,1.30000,126.000000,3.500000
50%,2004.000000,26.000000,6.000000,0.000000,0.000000,0.000000,62.000000,12.000000,1196.000000,19.900000,...,33.000000,0.600000,13.000000,0.300000,61.000000,1.100000,114.000000,1.90000,402.000000,6.700000
75%,2011.000000,30.000000,9.000000,0.000000,0.000000,0.000000,76.000000,53.000000,2035.000000,29.200000,...,62.000000,0.900000,33.000000,0.500000,113.000000,1.700000,173.000000,2.50000,807.000000,11.700000
max,2017.000000,44.000000,15.000000,1.000000,15.000000,15.000000,85.000000,83.000000,3533.000000,43.700000,...,246.000000,3.000000,376.000000,4.600000,464.000000,5.700000,371.000000,6.00000,2832.000000,35.400000


In [10]:
# Drop irrelevant features

data = data.drop('Player', axis=1)
data = data.drop('Year', axis=1)
# maybe drop age maybe not 
#averageData = averageData.drop('Age', axis=1)
data = data.drop('All NBA', axis=1)
data = data.drop('PER', axis=1)
data = data.drop('All NBA Acc', axis=1)
data.head()

,Age,Pos,All NBA Total Career,G,GS,MP,MPpG,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,26,3,0,82,82,2709,33.0,0.548,0.057,0.459,...,66,0.8,50,0.6,116,1.4,207,2.5,1061,12.9
1,27,3,0,82,21,2164,26.4,0.556,0.101,0.557,...,59,0.7,23,0.3,99,1.2,117,1.4,750,9.1
2,28,3,0,82,53,2902,35.4,0.556,0.070,0.569,...,91,1.1,36,0.4,111,1.4,141,1.7,1116,13.6
3,29,3,0,82,55,2819,34.4,0.603,0.065,0.531,...,88,1.1,39,0.5,116,1.4,149,1.8,1051,12.8
4,30,3,0,82,55,2825,34.5,0.555,0.038,0.391,...,70,0.9,38,0.5,100,1.2,142,1.7,1204,14.7


In [11]:
# Create correlation matrix

corr_matrix = data.corr().abs()

# Select upper triangle of correlation matrix

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

['FGA',
 'FGApG',
 '3PpG',
 '3PA',
 '3PApG',
 '2P',
 '2PA',
 'FTA',
 'FTApG',
 'TRB',
 'TRBpG',
 'PTS',
 'PTSpG']

In [12]:
# Drop the highly corelated features from dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    data = data.drop(feature, axis=1)

In [13]:
data.describe()

,Age,Pos,All NBA Total Career,G,GS,MP,MPpG,TS%,3PAr,FTr,...,AST,ASTpG,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG
count,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,...,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.000000,12219.00000
mean,26.949505,5.904166,0.496358,54.075211,26.483182,1280.981750,20.566208,0.507746,0.185184,0.315713,...,117.721008,1.844758,41.714379,0.666454,26.458384,0.422179,76.029790,1.229888,115.083313,1.91911
std,4.213603,4.284325,1.904681,24.976332,29.590902,925.777987,10.262107,0.086997,0.192845,0.215042,...,139.487692,1.849850,37.332904,0.469495,36.990469,0.508466,64.856059,0.799250,76.999472,0.83143
min,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,24.000000,3.000000,0.000000,35.000000,1.000000,425.500000,12.000000,0.478000,0.008000,0.197000,...,21.000000,0.600000,11.000000,0.300000,4.000000,0.100000,22.000000,0.600000,47.000000,1.30000
50%,26.000000,6.000000,0.000000,62.000000,12.000000,1196.000000,19.900000,0.519000,0.128000,0.284000,...,69.000000,1.200000,33.000000,0.600000,13.000000,0.300000,61.000000,1.100000,114.000000,1.90000
75%,30.000000,9.000000,0.000000,76.000000,53.000000,2035.000000,29.200000,0.552000,0.325000,0.391000,...,162.000000,2.500000,62.000000,0.900000,33.000000,0.500000,113.000000,1.700000,173.000000,2.50000
max,44.000000,15.000000,15.000000,85.000000,83.000000,3533.000000,43.700000,1.136000,1.000000,6.000000,...,1164.000000,14.500000,246.000000,3.000000,376.000000,4.600000,464.000000,5.700000,371.000000,6.00000


In [14]:
# Split dataframe into train and test sets aiming for ALL NBA TOTAL CAREER

X = data.drop("All NBA Total Career", axis=1)
y = data["All NBA Total Career"]

test_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(9775, 49) (9775,)
(2444, 49) (2444,)


In [15]:
# Train the linear regression algorithm using default parameters

r = LinearRegression()  
r.fit(X_train, y_train)
y_pred = r.predict(X_test)
df1 = pd.DataFrame({'Actual': y_test, 'Predicted All NBA': y_pred})
df1

,Actual,Predicted All NBA
12275,0,0.876013
14445,0,0.230918
2001,0,0.107178
11974,0,1.569036
14641,0,0.636580
11496,0,0.291471
9005,0,0.267299
4990,0,-0.373360
12132,0,-0.389635
4255,0,-0.220743


In [16]:
r.coef_

array([ 5.16068084e-02,  1.62160825e-02,  2.36636149e-02,  4.76648690e-03,
       -1.52480176e-03, -6.70937594e-02, -4.67537726e+00,  3.13178982e-01,
        8.37202112e-02, -1.10667177e-01, -1.10885140e-01,  2.02392772e-01,
       -2.07508276e-02,  5.40350574e-02, -1.13833354e-02,  1.59260497e-02,
       -1.11733408e-02, -4.20853407e-02,  1.56316159e-02,  1.46359021e-01,
        1.04630095e-01, -1.97081632e-02, -1.00873318e-01,  4.94360811e-02,
        3.63673179e-01,  3.07428216e-03,  2.07792873e-01, -3.34880954e-01,
       -5.81543819e-03, -1.51380917e-01, -6.99060461e-01,  3.30673894e+00,
       -4.71976102e-04,  3.74846358e-01, -5.32471368e-01, -8.02939527e-03,
        8.36435709e-02,  2.22553381e-03,  2.50680699e-01, -2.01778055e-04,
        2.73763735e-01, -8.16674874e-04, -1.91030091e-01,  2.44036913e-03,
        3.47631533e-01,  7.92465050e-04,  3.46665306e-02, -3.82898744e-03,
       -1.95738552e-02])

In [17]:
r2_score(y_test, y_pred) 

0.40200847050964195

In [18]:
mean_squared_error(y_test, y_pred)

2.220665918794734

In [65]:
dataCurrent = pd.read_csv('2019Data.csv')
dataCurrent.head()

,Year,Player,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,2019,Luka Doncic,20,9,0,0,0,72,72,2318,...,77,1.1,25,0.3,247,3.4,137,1.9,1526,21.2
1,2016,Karl-Anthony Towns,20,0,0,0,2,82,82,2627,...,58,0.7,138,1.7,183,2.2,245,3.0,1503,18.3
2,2017,Karl-Anthony Towns,21,0,0,0,2,82,82,3030,...,57,0.7,103,1.3,212,2.6,241,2.9,2061,25.1
3,2018,Karl-Anthony Towns,22,0,1,1,2,82,82,2918,...,64,0.8,115,1.4,159,1.9,285,3.5,1743,21.3
4,2019,Karl-Anthony Towns,23,0,1,2,2,77,77,2545,...,67,0.9,125,1.6,240,3.1,292,3.8,1880,24.4


In [66]:
# Drop irrelevant features

names = dataCurrent['Player']
dataCurrent = dataCurrent.drop('Player', axis=1)
years = dataCurrent['Year']
dataCurrent = dataCurrent.drop('Year', axis=1)
dataCurrent = dataCurrent.drop('PER', axis=1)
#Average2019 = Average2019.drop('Age', axis=1)
dataCurrent = dataCurrent.drop('All NBA', axis=1)
accAllNBA = dataCurrent['All NBA Acc']
dataCurrent = dataCurrent.drop('All NBA Acc', axis=1)
dataCurrent.head()

,Age,Pos,All NBA Total Career,G,GS,MP,MPpG,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,20,9,0,72,72,2318,32.2,0.545,0.433,0.409,...,77,1.1,25,0.3,247,3.4,137,1.9,1526,21.2
1,20,0,2,82,82,2627,32.0,0.590,0.076,0.239,...,58,0.7,138,1.7,183,2.2,245,3.0,1503,18.3
2,21,0,2,82,82,3030,37.0,0.618,0.186,0.289,...,57,0.7,103,1.3,212,2.6,241,2.9,2061,25.1
3,22,0,2,82,82,2918,35.6,0.646,0.243,0.343,...,64,0.8,115,1.4,159,1.9,285,3.5,1743,21.3
4,23,0,2,77,77,2545,33.1,0.622,0.278,0.343,...,67,0.9,125,1.6,240,3.1,292,3.8,1880,24.4


In [67]:
# Drop previously highly correlated features from new dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    dataCurrent = dataCurrent.drop(feature, axis=1)

In [68]:
X2019 = dataCurrent.drop('All NBA Total Career', axis=1)
y2019 = dataCurrent['All NBA Total Career']

In [69]:
print(X2019.shape)
print(y2019.shape)

(23, 49)
(23,)


In [70]:
print(y2019_pred - y2019)

0     3.805146
1     1.264138
2     2.700582
3     2.618310
4     3.112601
5     0.019211
6     0.595494
7     0.327312
8     0.369618
9    -0.285835
10    0.853720
11    1.709015
12    2.326947
13   -5.697084
14   -4.756759
15   -4.872384
16   -3.317068
17   -1.351177
18   -0.433876
19    0.909657
20   -1.596471
21   -0.999075
22   -1.896146
Name: All NBA Total Career, dtype: float64


In [71]:
# Use previous model to predict All NBA Total Career based on current career's stats

y2019_pred = r.predict(X2019)
df3 = pd.DataFrame({'Season': years, 'Player': names, 'Predicted All NBA': y2019_pred, 'Remaining': y2019_pred - accAllNBA})
df3

,Season,Player,Predicted All NBA,Remaining
0,2019,Luka Doncic,3.805146,3.805146
1,2016,Karl-Anthony Towns,3.264138,3.264138
2,2017,Karl-Anthony Towns,4.700582,4.700582
3,2018,Karl-Anthony Towns,4.618310,3.618310
4,2019,Karl-Anthony Towns,5.112601,3.112601
5,2012,Kyrie Irving,2.019211,2.019211
6,2013,Kyrie Irving,2.595494,2.595494
7,2014,Kyrie Irving,2.327312,2.327312
8,2015,Kyrie Irving,2.369618,1.369618
9,2016,Kyrie Irving,1.714165,0.714165


In [72]:
df3.loc[df3['Remaining'] < 0, 'Remaining'] = 0

In [73]:
df3.round()

,Season,Player,Predicted All NBA,Remaining
0,2019,Luka Doncic,4.0,4.0
1,2016,Karl-Anthony Towns,3.0,3.0
2,2017,Karl-Anthony Towns,5.0,5.0
3,2018,Karl-Anthony Towns,5.0,4.0
4,2019,Karl-Anthony Towns,5.0,3.0
5,2012,Kyrie Irving,2.0,2.0
6,2013,Kyrie Irving,3.0,3.0
7,2014,Kyrie Irving,2.0,2.0
8,2015,Kyrie Irving,2.0,1.0
9,2016,Kyrie Irving,2.0,1.0
